In [39]:
import numpy as np

In [40]:
# 对应状态集合Q
states = ('Healthy', 'Fever')
# 对应观测集合V
observations = ('normal', 'cold', 'dizzy')
# 初始状态概率向量π
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
# 状态转移矩阵A
transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}
# 观测概率矩阵B
emission_probability = {
    'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}

In [41]:
def generate_index_map(lables):
    id2label = {}
    label2id = {}
    i = 0
    for l in lables:
        id2label[i] = l
        label2id[l] = i
        i += 1
    return id2label, label2id
 
states_id2label, states_label2id = generate_index_map(states)
observations_id2label, observations_label2id = generate_index_map(observations)
print(states_id2label, states_label2id)
print(observations_id2label, observations_label2id)

{0: 'Healthy', 1: 'Fever'} {'Healthy': 0, 'Fever': 1}
{0: 'normal', 1: 'cold', 2: 'dizzy'} {'normal': 0, 'cold': 1, 'dizzy': 2}


In [42]:
def convert_map_to_vector(map_, label2id):
    """将概率向量从dict转换成一维array"""
    v = np.zeros(len(map_), dtype=float)
    for e in map_:
        v[label2id[e]] = map_[e]
    return v

 
def convert_map_to_matrix(map_, label2id1, label2id2):
    """将概率转移矩阵从dict转换成矩阵"""
    m = np.zeros((len(label2id1), len(label2id2)), dtype=float)
    for line in map_:
        for col in map_[line]:
            m[label2id1[line]][label2id2[col]] = map_[line][col]
    return m

In [43]:
A = convert_map_to_matrix(transition_probability, states_label2id, states_label2id)
print(A)
B = convert_map_to_matrix(emission_probability, states_label2id, observations_label2id)
print(B)
observations_index = [observations_label2id[o] for o in observations]
pi = convert_map_to_vector(start_probability, states_label2id)
print(pi)

[[0.7 0.3]
 [0.4 0.6]]
[[0.5 0.4 0.1]
 [0.1 0.3 0.6]]
[0.6 0.4]


In [44]:
# 随机生成观测序列和状态序列    
def simulate(T):

    def draw_from(probs):
        """
        1.np.random.multinomial:
        按照多项式分布，生成数据
        >>> np.random.multinomial(20, [1/6.]*6, size=2)
                array([[3, 4, 3, 3, 4, 3],
                       [2, 4, 3, 4, 0, 7]])
         For the first run, we threw 3 times 1, 4 times 2, etc.  
         For the second, we threw 2 times 1, 4 times 2, etc.
        2.np.where:
        >>> x = np.arange(9.).reshape(3, 3)
        >>> np.where( x > 5 )
        (array([2, 2, 2]), array([0, 1, 2]))
        """
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

    observations = np.zeros(T, dtype=int)
    states = np.zeros(T, dtype=int)
    states[0] = draw_from(pi)
    observations[0] = draw_from(B[states[0],:])
    for t in range(1, T):
        states[t] = draw_from(A[states[t-1],:])
        observations[t] = draw_from(B[states[t],:])
    return observations, states

In [45]:
# 生成模拟数据
observations_data, states_data = simulate(10)
print(observations_data)
print(states_data)
# 相应的label
print("病人的状态: ", [states_id2label[index] for index in states_data])
print("病人的观测: ", [observations_id2label[index] for index in observations_data])

[2 1 1 1 0 1 1 1 1 1]
[1 0 0 1 1 0 0 0 1 1]
病人的状态:  ['Fever', 'Healthy', 'Healthy', 'Fever', 'Fever', 'Healthy', 'Healthy', 'Healthy', 'Fever', 'Fever']
病人的观测:  ['dizzy', 'cold', 'cold', 'cold', 'normal', 'cold', 'cold', 'cold', 'cold', 'cold']


## offline 

In [46]:
def forward(obs_seq):
    """前向算法"""
    N = A.shape[0]
    T = len(obs_seq)
    
    # F保存前向概率矩阵
    F = np.zeros((N,T))
    F[:,0] = pi * B[:, obs_seq[0]]

    for t in range(1, T):
        for n in range(N):
            F[n,t] = np.dot(F[:,t-1], (A[:,n])) * B[n, obs_seq[t]]

    return F

def backward(obs_seq):
    """后向算法"""
    N = A.shape[0]
    T = len(obs_seq)
    # X保存后向概率矩阵
    X = np.zeros((N,T))
    X[:,-1:] = 1
#     print("x: ", X)

    for t in reversed(range(T-1)):
        for n in range(N):
#             dd = X[:,t+1] * A[n,:] * B[:, obs_seq[t+1]]
#             print(dd)
            X[n,t] = np.sum(X[:,t+1] * A[n,:] * B[:, obs_seq[t+1]])

    return X

In [47]:
forward(observations_data)

array([[6.00000000e-02, 5.52000000e-02, 2.32320000e-02, 8.69952000e-03,
        3.95678400e-03, 1.19343091e-03, 4.06533996e-04, 1.44042125e-04,
        5.16241537e-05, 1.85615942e-05],
       [2.40000000e-01, 4.86000000e-02, 1.37160000e-02, 4.55976000e-03,
        5.34571200e-04, 4.52333376e-04, 1.88828790e-04, 7.05772418e-05,
        2.56676948e-05, 9.26635889e-06]])

In [48]:
backward(observations_data)

array([[1.02258767e-04, 2.85331051e-04, 8.05375464e-04, 2.36504619e-03,
        6.27879520e-03, 1.74383200e-02, 4.84120000e-02, 1.34200000e-01,
        3.70000000e-01, 1.00000000e+00],
       [9.03851129e-05, 2.48511916e-04, 6.64732452e-04, 1.59069478e-03,
        5.58226240e-03, 1.55118400e-02, 4.31440000e-02, 1.20400000e-01,
        3.40000000e-01, 1.00000000e+00]])

In [49]:
def baum_welch_train(observations, A, B, pi, criterion=0.05):
    """无监督学习算法——Baum-Weich算法"""
    n_states = A.shape[0]
    n_samples = len(observations)

    done = False
    while not done:
        # alpha_t(i) = P(O_1 O_2 ... O_t, q_t = S_i | hmm)
        # Initialize alpha
        alpha = forward(observations)

        # beta_t(i) = P(O_t+1 O_t+2 ... O_T | q_t = S_i , hmm)
        # Initialize beta
        beta = backward(observations)
        # ξ_t(i,j)=P(i_t=q_i,i_{i+1}=q_j|O,λ)
        xi = np.zeros((n_states,n_states,n_samples-1))
        for t in range(n_samples-1):
            denom = np.dot(np.dot(alpha[:,t].T, A) * B[:,observations[t+1]].T, beta[:,t+1])
            for i in range(n_states):
                numer = alpha[i,t] * A[i,:] * B[:,observations[t+1]].T * beta[:,t+1].T
                xi[i,:,t] = numer / denom

        # γ_t(i)：gamma_t(i) = P(q_t = S_i | O, hmm)
        gamma = np.sum(xi,axis=1)
        # Need final gamma element for new B
        # xi的第三维长度n_samples-1，少一个，所以gamma要计算最后一个
        prod =  (alpha[:,n_samples-1] * beta[:,n_samples-1]).reshape((-1,1))
        gamma = np.hstack((gamma,  prod / np.sum(prod))) #append one more to gamma!!!
        
        # 更新模型参数
        newpi = gamma[:,0]
        newA = np.sum(xi,2) / np.sum(gamma[:,:-1],axis=1).reshape((-1,1))
        newB = np.copy(B)
        num_levels = B.shape[1]
        sumgamma = np.sum(gamma,axis=1)
        for lev in range(num_levels):
            mask = observations == lev
            newB[:,lev] = np.sum(gamma[:,mask],axis=1) / sumgamma
        
        # 检查是否满足阈值
        if np.max(abs(pi - newpi)) < criterion and \
                        np.max(abs(A - newA)) < criterion and \
                        np.max(abs(B - newB)) < criterion:
            done = 1
        A[:], B[:], pi[:] = newA, newB, newpi
    return newA, newB, newpi

In [50]:
A = np.array([[0.5, 0.5],[0.5, 0.5]])
B = np.array([[0.3, 0.3, 0.4],[0.3, 0.3, 0.4]])
pi = np.array([0.4, 0.6])
print("pi: ", pi)
pic = pi.copy()

observations_data, states_data = simulate(200)
newA, newB, newpi = baum_welch_train(observations_data, A, B, pic,0.02)
print("newA: ", newA)
print("newB: ", newB)
print("newpi: ", newpi)
print("pi: ", pi)


pi:  [0.4 0.6]
newA:  [[0.50000658 0.49999342]
 [0.50000735 0.49999265]]
newB:  [[0.32064046 0.26352894 0.4158306 ]
 [0.31936082 0.26646812 0.41417106]]
newpi:  [0.39866941 0.60133059]
pi:  [0.4 0.6]


## online
Online Learning with Hidden Markov Models[ Gianluigi Mongillo ,Sophie Deneve]

Start with: 
- initial guess for the model parameters, θ(0) = [A,B], 
- initial state probabilities, ˆql (0) ≡ P(x0 = l), 
- sufficient statistics, ˆφh ijk(0)

specific data struct


In [1]:
import numpy as np

In [2]:
# 初始状态概率向量π
pi = np.array([0.5, 0.5])
print("初始状态概率向量π: ", pi)

初始状态概率向量π:  [0.5 0.5]


In [3]:
# 初始状态转移矩阵和观测矩阵
A = np.array([[0.5, 0.5],[0.5, 0.5]])
B = np.array([[0.333, 0.333, 0.333],[0.333, 0.333, 0.333]])
print("初始状态转移矩阵A:\n", A)
print("初始观测矩阵A:\n", B)


初始状态转移矩阵A:
 [[0.5 0.5]
 [0.5 0.5]]
初始观测矩阵A:
 [[0.333 0.333 0.333]
 [0.333 0.333 0.333]]


In [4]:
# P(xt−1 = i, xt = j, xT−1 = l, xT = h, y0→T)
# which element to update in transition model or observation mode
# i = 1 # xt-1
# j = 0 # xt

# observed  
# k = 0 #yT

# last two state
# l = 0 # xT-1
# h = 0 # xT

In [5]:
time_factor = 0.001
print("time_factor: ", time_factor)

time_factor:  0.001


In [6]:
def delta(i, j):
    if i == j:
        return 1
    else :
        return 0
def g_delta(i, j, l, h):
    return delta(i, l )*delta(j, h)

In [7]:
# γlh(yT) 
# probility P(xT-1 = l, xT = h | yT = k)
Gama = 0.0*np.ones([2,2,3])
print("Gama:\n", Gama)

Gama:
 [[[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]]


In [8]:
# 状态概率 ql (T − 1) ≡ P(xT−1 = l|y0→T−1)
# probility P(xT-1 = l | y0-T-1)
Q = pi
print("状态概率：\n", Q)

状态概率：
 [0.5 0.5]


In [9]:
# 信息概率
# P(xt-1 = i, xt = j, xT = h , yT = h)
Fi = 0.33*np.ones([2,2,2,3])
print("信息概率:\n ", Fi)

信息概率:
  [[[[0.33 0.33 0.33]
   [0.33 0.33 0.33]]

  [[0.33 0.33 0.33]
   [0.33 0.33 0.33]]]


 [[[0.33 0.33 0.33]
   [0.33 0.33 0.33]]

  [[0.33 0.33 0.33]
   [0.33 0.33 0.33]]]]


In [10]:
# initialise all probility with A B
Q0 = Q
print("Q0\n", Q0)

Q0
 [0.5 0.5]


In [11]:
# update after get observation
# get new obs
k = 0

# update Gama(T) from Q(T-1)
# Gama(l, h, k)
prob_sum = (A[0][0]*B[0][k]*Q[0] + A[0][1]*B[1][k]*Q[0] + A[1][0]*B[0][k]*Q[1] + A[1][1]*B[1][k]*Q[1]  )
Gama[0][0][k] = (A[0][0]*B[0][k]) / prob_sum
Gama[0][1][k] = (A[0][1]*B[1][k]) / prob_sum
Gama[1][0][k] = (A[1][0]*B[0][k]) / prob_sum
Gama[1][1][k] = (A[1][1]*B[1][k]) / prob_sum


# update internal state posibility
# Q(T) = Gama(T)*Q(T-1)

Q[0]
Q[0] = (Gama[0][0][k] * Q0[0]) + (Gama[1][0][k] * Q0[1])  
Q[1] = (Gama[0][1][k] * Q0[0]) + (Gama[1][1][k] * Q0[1])  
print("Q:\n", Q)

# P(xt-1 = i, xt = j, xT = h , yT = k)
# updata enven corespond yT is not observed
# Fi[0][0][0][0] = 0.0
# Fi[i][j][h][k] = Gama[l][h][k] * Fi[i][j][l][k] G(h,l) Q(l)
Fi[0][0][0][0]  = (Gama[0][0][k] * (Fi[0][0][0][0] + time_factor*(delta(0, k)*g_delta(0, 0 ,0, 0)*Q[0] - Fi[0][0][0][0] ) ) ) + \
     (Gama[1][0][k] * (Fi[0][0][1][0] + time_factor*(delta(0, k)*g_delta(0, 0 ,1, 0)*Q[1] - Fi[0][0][1][0] ) ) )

print("Fi(0,0,0,0)\n", Fi[0][0][0][0] )

Q:
 [0.5 0.5]
Fi(0,0,0,0)
 0.32992


In [ ]:
Fi

In [ ]:
Gama,Q,

In [ ]:
for i in range(100):
    
    Fi[0][0][0][0]  = (Gama[0][0][k] * (Fi[0][0][0][0] + time_factor*(delta(0, k)*g_delta(0, 0 ,0, 0)*Q[0] - Fi[0][0][0][0] ) ) ) + \
         (Gama[1][0][k] * (Fi[0][0][1][0] + time_factor*(delta(0, k)*g_delta(0, 0 ,1, 0)*Q[1] - Fi[0][0][1][0] ) ) )

print("Fi(0,0,0,0)", Fi[0][0][0][0] )

In [11]:
class Q_data:
    def __init__(self,Q_):
        self.Q = Q_
    
    def updateQ(self, new_obs_k, Gama_):
        Q0 = self.Q
        print("K: ", new_obs_k, "Gama: ", Gama_, "Q0: ", Q0)
        print("debug: ",Gama_[0][0][new_obs_k] ,", ", Q0[0],", ", Gama_[1][0][new_obs_k] ,",",  Q0[1]   )
        self.Q[0] = (Gama_[0][0][new_obs_k] * Q0[0]) + (Gama_[1][0][new_obs_k] * Q0[1])  

        self.Q[1] = (Gama_[0][1][new_obs_k] * Q0[0]) + (Gama_[1][1][new_obs_k] * Q0[1])  
        
        print("updated Q = \n", self.Q)

    def getQ(self):
        return self.Q

In [12]:
class Gama_data:
    def __init__(self, Gama_):
        self.Gama = Gama_
        self.prob_sum = None
        self.A = None
        self.B = None
    
    def updateModel(self, A_, B_):
        self.A = A_
        self.B = B_
        
    def updateGama(self, new_obs_k, Q_old):
        A_ = self.A
        B_ = self.B
        Q_ = Q_old
        prob_sum = (A_[0][0]*B_[0][new_obs_k]*Q_[0] + A_[0][1]*B_[1][new_obs_k]*Q_[0]\
                  + A_[1][0]*B_[0][new_obs_k]*Q_[1] + A_[1][1]*B_[1][new_obs_k]*Q_[1]  )
        self.prob_sum = prob_sum
        print (" (A[0][0]*B[0][k]) / prob_sum", (A_[0][0]*B_[0][new_obs_k]) , prob_sum)
        self.Gama[0][0][new_obs_k] = (A_[0][0]*B_[0][new_obs_k]) / prob_sum

        self.Gama[0][1][new_obs_k] = (A_[0][1]*B_[1][new_obs_k]) / prob_sum

        self.Gama[1][0][new_obs_k] = (A_[1][0]*B_[0][new_obs_k]) / prob_sum

        self.Gama[1][1][new_obs_k] = (A_[1][1]*B_[1][new_obs_k]) / prob_sum
        
    
    
    def get(self, i_, j_, k_):
        return self.Gama[i_][j_][k_]
        
    def getGama(self):
        return self.Gama
    
# how to define a datastruct
# member data
# method

# for Gama
# contatin latest Gama
# updata Gama from Q and yT, A, B
        

In [13]:
m_gama = Gama_data(Gama)

In [15]:
m_gama.updateModel(A,B)
m_gama.updateGama(new_obs_k=0,Q_old=Q0)

 (A[0][0]*B[0][k]) / prob_sum 0.1665 0.333


In [16]:
Q0

array([0.5, 0.5])

In [17]:
m_gama.getGama()

array([[[0.5, 0. , 0. ],
        [0.5, 0. , 0. ]],

       [[0.5, 0. , 0. ],
        [0.5, 0. , 0. ]]])

In [18]:
m_q = Q_data(Q0)

In [19]:
m_q.getQ()

array([0.5, 0.5])

In [20]:
m_q.updateQ(0, m_gama.getGama())

K:  0 Gama:  [[[0.5 0.  0. ]
  [0.5 0.  0. ]]

 [[0.5 0.  0. ]
  [0.5 0.  0. ]]] Q0:  [0.5 0.5]
debug:  0.5 ,  0.5 ,  0.5 , 0.5
updated Q = 
 [0.5 0.5]


In [21]:
m_q.getQ()

array([0.5, 0.5])

In [22]:
Q

array([0.5, 0.5])

In [29]:
class Fi_data:
    def __init__(self, Fi_, time_factor_):
        self.Fi = Fi_
        self.time_factor = time_factor_
        
    def updateMode(self,new_obs_k ,Gama_, Q_):
        for i in range(self.Fi.shape[0]):
            for j in range(self.Fi.shape[1]):
                for h in range(self.Fi.shape[2]):
                    for k in range(self.Fi.shape[3]):
                        self.Fi[i][j][h][k]  = (Gama_[0][h][k] * (self.Fi[i][j][0][k] + self.time_factor*(delta(new_obs_k, k)*g_delta(i, j ,0, h)*Q_[0] - self.Fi[i][j][0][k] ) ) ) + \
                                               (Gama_[1][h][k] * (self.Fi[i][j][1][k] + self.time_factor*(delta(new_obs_k, k)*g_delta(i, j ,1, h)*Q_[1] - self.Fi[i][j][1][k] ) ) )
                
    def getFi(self):
        return self.Fi;

array([[[[0.32992   , 0.        , 0.        ],
         [0.32963004, 0.        , 0.        ]],

        [[0.32967   , 0.        , 0.        ],
         [0.32975517, 0.        , 0.        ]]],


       [[[0.32992   , 0.        , 0.        ],
         [0.32963004, 0.        , 0.        ]],

        [[0.32967   , 0.        , 0.        ],
         [0.32975517, 0.        , 0.        ]]]])

In [31]:

m_fi = Fi_data(Fi, time_factor)

In [32]:
m_fi.updateMode(0, m_gama.getGama(), m_q.getQ())

In [33]:
m_fi.getFi()

array([[[[0.32969524, 0.        , 0.        ],
         [0.32933298, 0.        , 0.        ]],

        [[0.32938287, 0.        , 0.        ],
         [0.32948945, 0.        , 0.        ]]],


       [[[0.32969524, 0.        , 0.        ],
         [0.32933298, 0.        , 0.        ]],

        [[0.32938287, 0.        , 0.        ],
         [0.32948945, 0.        , 0.        ]]]])

In [ ]:



b1 = Fi[:,1,:,1].sum() /  Fi[:,1,:,:].sum()
print("b1: ", b1)

In [ ]:
l = 0
h = 1

In [ ]:
Gama = np.ones([2,2])

In [ ]:
Gama[0][0] = A[0][1] * B[0][1] / (A[0][1] * B[1][1]*Q[0] + A[0][0] * B[0][1]*Q[0] + A[1][0] * B[0][1]*Q[1] + A[1][1] * B[1][1]*Q[1] )

In [ ]:
Gama

In [ ]:
f1 = Fi[0][0][0][0]

In [ ]:
f1 = (Gama[0] + Gama[1]) * (f1 + time_factor* )

In [ ]:
Fi[0][0][0][0] = (Gama[0] *  ) + ()

set state dim, observation dim
state_dim = 2
obs_dim = 3


In [70]:
class HMM:
    def __init__(self, state_dim_, obs_dim_):
        self.A = np.ones([state_dim_, state_dim_]) / float(state_dim_)
        self.B = np.ones([state_dim_, obs_dim_])/ float(obs_dim_)
        self.pi = np.ones([state_dim_])/ float(state_dim_)
        
        print("A:\n", self.A, "\nB:\n", self.B, "\npi:\n", self.pi)
        
        Gama_ = 0.0*np.ones([2,2,3])

        print("Gama:\n", Gama_)
        
        self.gama = Gama_data(Gama_)
        
        Q_ = pi
        
        self.q = Q_data(Q_)
        
        Fi_ = np.ones([state_dim_,state_dim_,state_dim_,obs_dim_])/float(obs_dim_)
        print("信息概率:\n ", Fi_)
        
        time_factor_ = 0.001
        
        self.fi = Fi_data(Fi_, time_factor_)
        
        
#         self.Q 
#         self.Gama
    def learn(self, data_):
        
        print("get data:\n", data_)
        
        for ob in data_:
            
#             updata gama
            self.gama.updateModel(self.A,self.B)
            self.gama.updateGama(new_obs_k=ob,Q_old=self.q.getQ())



#             updata q
            self.q.updateQ(0, self.gama.getGama())

#             update fi
            self.fi.updateMode(0, self.gama.getGama(), self.q.getQ())


            

In [71]:
state_dim = 2
obs_dim = 3
hmm = HMM(state_dim, obs_dim)

A:
 [[0.5 0.5]
 [0.5 0.5]] 
B:
 [[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]] 
pi:
 [0.5 0.5]
Gama:
 [[[0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]]]
信息概率:
  [[[[0.33333333 0.33333333 0.33333333]
   [0.33333333 0.33333333 0.33333333]]

  [[0.33333333 0.33333333 0.33333333]
   [0.33333333 0.33333333 0.33333333]]]


 [[[0.33333333 0.33333333 0.33333333]
   [0.33333333 0.33333333 0.33333333]]

  [[0.33333333 0.33333333 0.33333333]
   [0.33333333 0.33333333 0.33333333]]]]


In [72]:
print("ground truth,observations_data\n",observations_data, "\nstates_data\n ",states_data)

ground truth,observations_data
 [1 0 2 1 1 2 2 2 0 1 2 0 2 0 1 0 2 2 1 0 2 2 1 1 0 2 0 2 1 2 2 0 0 2 0 1 2
 1 2 1 2 0 2 2 0 0 0 1 1 0 1 0 0 2 2 2 0 0 2 0 0 1 2 0 1 0 1 0 0 0 0 1 1 1
 0 2 0 2 0 0 2 0 0 0 1 1 2 0 0 0 2 0 1 1 2 2 1 1 0 1 1 2 2 2 1 2 2 0 2 2 2
 0 1 2 1 1 2 2 2 2 2 0 2 2 2 2 1 2 1 0 2 0 2 2 2 0 2 2 0 2 0 2 1 1 1 1 2 1
 2 2 0 1 2 1 2 0 0 1 0 2 2 2 1 2 1 2 2 2 2 2 2 0 0 2 0 1 1 2 0 0 2 1 1 1 0
 2 2 0 2 0 0 1 2 0 2 0 1 1 0 2] 
states_data
  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 0 1
 1 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 1 0
 1 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0
 0 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0
 0 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 1 0 1
 1 1 1 0 0 1 1 1 1 0 0 0 1 0 1]


In [74]:
t= np.array([0])
hmm.learn(t)

get data:
 [0]
 (A[0][0]*B[0][k]) / prob_sum 0.16666666666666666 nan
K:  0 Gama:  [[[nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]]] Q0:  [nan nan]
debug:  nan ,  nan ,  nan , nan
updated Q = 
 [nan nan]
